[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gamedaygeorge/ceos-db-toolkit/blob/8db8cdd2c6d9f5899f4361ed1875164f2ab931ca/colab-notebooks/Timelines_measurements_wavebands_orbits.ipynb)

## Background

Script to export CEOS Database mission metadata in a SKOS format. You can read more about [SKOS here](https://www.w3.org/2004/02/skos/).

The script is intended as a proof of concept. Further refinements could be made both to the content and format of the SKOS information, as well as to the process overall.

Running the script requires access to the CEOS Database. This access information is stored in a YAML file as specified in the notebook. A request for this information can be sent to ceosmim@gmail.com.

In [1]:
# Import API-related modules
import requests
import json

# Import for Quick Gantt Chart with Matplotlib example.
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.font_manager as font_manager
import matplotlib.dates
from matplotlib.dates import WEEKLY,MONTHLY,YEARLY, DateFormatter, rrulewrapper, RRuleLocator
import numpy as np

# Import to sort mission dates.
from operator import itemgetter, attrgetter

#Import for date conversion.
from dateutil import parser
import datetime
import time

#Import for SQL tables
import pandas as pd
from pandas import DataFrame

In [2]:
# Script specific imports
import sys, getopt
import argparse
import textwrap

import xml.etree.ElementTree as ET
from datetime import datetime

import csv

## Connect to Google Drive and the database

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Specify the path to yml file contating database access information

In [4]:
# Specify the path in your local drive (use !ls and !pwd to hunt for it)
#Libby path:config_path='/content/drive/MyDrive/MIM Colab/config-libby.yml'
#George path:
config_path='/content/drive/MyDrive/Colab-MIM Notebooks/config.yml'
print('Getting CEOS Dev DB Config from: ' + config_path)

Getting CEOS Dev DB Config from: /content/drive/MyDrive/Colab-MIM Notebooks/config.yml


#### Save access information imported from .yml file


In [5]:
# Define SECRET database connection variables (MIM Dev DB on AWS RDS)
# Like YAML here: https://veekaybee.github.io/2020/02/25/secrets/
import yaml

with open(config_path, 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

user = cfg['ceos_dev_db']['user']
password = cfg['ceos_dev_db']['password']
database = cfg['ceos_dev_db']['database']
server = cfg['ceos_dev_db']['server']

#### Install DB Access Layer

A few code blocks to setup the necessary software to access the CEOS DB.

Define basic functions to access CEOS DB.

In [6]:
# CHECK YOUR ODC COLAB VIRTUAL MACHINE IP ADDRESS
# The CEOS Dev DB is protected by whitelisting due to lots of attempts to query it
# Whitelisting means you can only connect from certain IP addresses

# Used this command to find the IP address of the Colab notebook (VM)
!curl ifconfig.me

# THE FOLLOWING RANGES HAVE BEEN WHITELISTED TO CONNECT TO THE AWS DATABASE
# 34.xxx.xxx.xxx
# 35.xxx.xxx.xxx
# 104.xxx.xxx.xxx

# IF YOUR VM IP IS NOT IN THIS RANGE* THEN IT WILL HAVE TO BE ADDED TO THE WHITELIST BEFORE IT WILL WORK (CONTACT GEORGE)
# * As a work around, you could start a new Colab notebook (VM) to see if you can get an IP address in the whitelist range.

104.198.244.164

In [7]:
# Run this block to install the necessary DB interface code

%%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/21.10/prod.list > /etc/apt/sources.list.d/mssql-release.list

sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

OK
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://packages.microsoft.com/ubuntu/21.10/prod impish InRelease [3,638 B]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,031 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 https://packages.microsoft.com/ubuntu/21.10/prod i

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0   2993      0 --:--:-- --:--:-- --:--:--  2987
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    89  100    89    0     0    271      0 --:--:-- --:--:-- --:--:--   272
W: https://packages.microsoft.com/ubuntu/21.10/prod/dists/impish/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot b

In [8]:
# Run this block to install the Python library to connect to the DB
!pip install pyodbc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.0/336.0 kB 5.2 MB/s eta 0:00:00


In [9]:
# This this block to import the 'pyodbc' library
# pyodbc is used to connect Python (notebook) to the CEOS DB
# This import statment is how we include it when we're working on a local server
import pyodbc

In [10]:
# Function to get DB cursor return
def get_db_cursor(user, password, database, server, t_sql):
  conn = pyodbc.connect(DRIVER = '{ODBC Driver 17 for SQL Server}',
                      SERVER = server,
                      DATABASE = database,
                      UID = user,
                      PWD = password)

  db_cursor = conn.cursor()
  db_cursor.execute(t_sql)

  return db_cursor

# Define Functions

# Execution Block

In [11]:
import pyodbc
import json

# Function to fetch data from the database and create SKOS JSON-LD
def fetch_data_and_create_skos_jsonld():
    # SQL query to fetch measurement data with updated join logic
    t_sql = '''
    SELECT
        TypesMeasurementsWMO.measurementTypeWMOID,
        TypesMeasurementsWMO.measurementTypeWMONameFull,
        TypesMeasurementsWMO.measurementTypeWMODescription,
        TypesMeasurementsWMO.isAtmospheric,
        TypesMeasurements.measurementTypeID,
        TypesMeasurements.measurementTypeNameFull,
        CategoriesMeasurements.measurementCategoryID,
        CategoriesMeasurements.measurementCategoryNameFull,
        GCOS200ECVs.gcos200ECVID,
        GCOS200ECVs.gcos200ECVNameFull
    FROM GCOS200ECVs
    RIGHT OUTER JOIN TypesMeasurementsWMO
        ON GCOS200ECVs.gcos200ECVID = TypesMeasurementsWMO.gcos200ECVID
    LEFT OUTER JOIN CategoriesMeasurements
        INNER JOIN TypesMeasurements
        ON CategoriesMeasurements.measurementCategoryID = TypesMeasurements.measurementCategoryID
        ON TypesMeasurementsWMO.measurementTypeID = TypesMeasurements.measurementTypeID;
    '''

    # Fetch data from the database
    measurements = get_db_cursor(user, password, database, server, t_sql)

    # Create the SKOS structure
    skos_data = {
        "@context": {
            "skos": "http://www.w3.org/2004/02/skos/core#",
            "dc": "http://purl.org/dc/elements/1.1/"
        },
        "@graph": []
    }

    # Loop through each measurement row and create SKOS concepts
    for row in measurements:
        (measurementTypeWMOID, measurementTypeWMONameFull, measurementTypeWMODescription, isAtmospheric,
         measurementTypeID, measurementTypeNameFull, measurementCategoryID, measurementCategoryNameFull,
         gcos200ECVID, gcos200ECVNameFull) = row

        # Create the concept with the main measurement (for instruments, use measurementTypeWMOID in the URL)
        concept = {
            "@type": "skos:Concept",
            "@id": f"https://database.eohandbook.com/measurements/instruments.aspx?measurementTypeWMOID={measurementTypeWMOID}",
            "skos:prefLabel": measurementTypeWMONameFull,
            "skos:definition": measurementTypeWMODescription,
            "skos:broader": [
                {
                    "@type": "skos:Concept",
                    "@id": f"https://database.eohandbook.com/measurements/measurements.aspx?measurementTypeID={measurementTypeID}",
                    "skos:prefLabel": measurementTypeNameFull,
                    "skos:broader": {
                        "@type": "skos:Concept",
                        "@id": f"https://database.eohandbook.com/measurements/categories.aspx?measurementCategoryID={measurementCategoryID}",
                        "skos:prefLabel": measurementCategoryNameFull
                    }
                }
            ]
        }

        # Add GCOS ECV as a parallel broader concept if it exists
        if gcos200ECVID:
            concept["skos:broader"].append({
                "@type": "skos:Concept",
                "skos:prefLabel": f"{gcos200ECVNameFull} (GCOS ECV)"
            })

        # Append the concept to the graph
        skos_data["@graph"].append(concept)

    # Return SKOS data in JSON-LD format
    return json.dumps(skos_data, indent=4)

# Example usage
skos_jsonld = fetch_data_and_create_skos_jsonld()
print(skos_jsonld)


{
    "@context": {
        "skos": "http://www.w3.org/2004/02/skos/core#",
        "dc": "http://purl.org/dc/elements/1.1/"
    },
    "@graph": [
        {
            "@type": "skos:Concept",
            "@id": "https://database.eohandbook.com/measurements/instruments.aspx?measurementTypeWMOID=1",
            "skos:prefLabel": "Atmospheric temperature (column/profile)",
            "skos:definition": "Vertical profile of the atmospheric temperature - Requested from surface to TOA (layers: LT, HT, LS, HS&M) - Physical unit: [ K ] - Accuracy unit: [ K ].",
            "skos:broader": [
                {
                    "@type": "skos:Concept",
                    "@id": "https://database.eohandbook.com/measurements/measurements.aspx?measurementTypeID=25",
                    "skos:prefLabel": "Atmospheric Temperature Fields",
                    "skos:broader": {
                        "@type": "skos:Concept",
                        "@id": "https://database.eohandbook.com/measur